# Lösung: Generierung der Exportformate aus dem Bibsonomy-Suchergebnis

In [ ]:
import json
import requests

url = "https://www.bibsonomy.org/json/search/Bibliothek?items=1000&duplicates=merged"

result = requests.get(url)
result # Ausgabe des Ergebnisses - gibt Typ (Response) und HTTP-Statuscode (idealerweise 200) aus

In [ ]:
data = result.json()
data # Ansicht der Ausgabe

In [ ]:
items = data['items']

pubList = [] # neue leere Liste
for item in items:
    if item['type']=='Publication': # Bedingung für Aufnahme des Items in die neue Liste
        pubList.append(item) # item an Liste anhängen

#mit list comprehension:
#pubList = [item for item in items if item['type'] == 'Publication']

In [ ]:
pubList[0] # ein Item der Ergebnisliste mal zum Prüfen ausgeben

## Generieren des XML-Exportformats

Die beispielhafte XML-Struktur sieht so aus:

```
<publications>
    <publication>
        <id>1</id>
        <title>...</title>
        <authors>
            <author>...</author>
            <author>...</author>
        </authors>
        <abstract>...</abstract>
        <publisher>...</publisher>
        <year>...</year>
        <type>...</type>
        <!-- weitere Felder -->
    </publication>
    <publication>
    ...
    </publication>
    <!-- usw. -->
</publications>

```

Die Idee der ID könnte sein, statt der bibsonomy-internen ID eine neue zu verwenden, die für jeden Datensatz hochgezählt wird.

In [ ]:
# Einschub: String-Konkatenation
a = '<tag>'
b = 'hallo'
c = '</tag>'

concatenated = a+b+c # mit + lassen sich Strings einfach zusammenfügen
print(concatenated)

In [ ]:
with open('output/bibsonomy-manuell.xml', 'w', encoding="utf-8") as outfile:
    
    # Zunächst muss das Wurzelelement geschrieben werden
    outfile.write('<publications>\n')
    
    # Nun iterieren wir über unsere Datensätze
    for index, data in enumerate(pubList): # enumerate gibt Tupel (index, item) zurück
        outfile.write('\t<publication>\n')
        
        index_str = str(index) # zunächst die Zahl zum String umwandeln
        
        outfile.write('\t\t<id>' + index_str + '</id>\n') # Index-String mit den anderen Zeichenfolgen verbinden
        
        # Bereinigen des Titels
        title = data['label']
        title = title.replace("<", "&lt;")
        title = title.replace(">", "&gt;")
        title = title.replace("&", "&amp;")
        
        outfile.write('\t\t<title>' + title + '</title>\n')
        
        outfile.write('\t\t<authors>\n')
        if 'author' in data:
            for author in data['author']:
                outfile.write('\t\t\t<author>' + author + '</author>\n')
        outfile.write('\t\t</authors>\n')
        
        outfile.write('\t\t<abstract>')
        if 'abstract' in data:
            # Bereinigen des Abstract
            abstract = data['abstract']
            abstract = abstract.replace("\n\n", " ")
            abstract = abstract.replace("<", "&lt;")
            abstract = abstract.replace(">", "&gt;")
            abstract = abstract.replace("&", "&amp;")
            
            outfile.write(abstract)
        outfile.write('</abstract>\n')
        
        outfile.write('\t\t<publisher>')
        if 'publisher' in data:
            publisher = data['publisher'].replace("&", "&amp;")
            outfile.write(publisher)
        outfile.write('</publisher>\n')
            
        outfile.write('\t\t<year>' + data['year'] + '</year>\n')
        
        outfile.write('\t\t<type>' + data['pub-type'] + '</type>\n')
        
        outfile.write('\t</publication>\n')
        
    # Am Schluss wird das Wurzelelement geschlossen
    outfile.write('</publications>')

### Erläuterungen

* wir verwenden hier Steuerzeichen für Zeilenumbrüche (`\n`) und Tabulatoren (`\t`) lediglich, damit der Output am Ende besser lesbar ist
* die Zeichen <, > und & sind in XML bereits reserviert (wie einige andere) - entsprechend müssen eventuelle Vorkommen in Strings zuvor bereinigt werden. Die `replace` Funktion ersetzt alle Vorkommen des ersten Parameters durch den zweiten. (Es würde sich anbieten, hierfür eine separate Funktion zu schreiben, die alle Bereinigungen durchführt.)
* nicht jeder unserer Datensätze enthält alle gefragten Daten. Daher sind mitunter `if`-Abfragen nötig, damit es nicht zu einem `KeyError` beim Zugriff auf nicht existente Schlüssel gibt.

#### Einschub: Übersicht über vorhandene Keys erstellen

Einfache Übersicht: Durch Iteration über den Datensatz eine Liste der darin vorhandenen Keys erstellen.

In [ ]:
keyList = []

for elem in pubList:
    for key in elem.keys():
        if key not in keyList: # wenn der Key bisher nicht in der Liste ist
            keyList.append(key)
            
len(keyList) # wie viele verschiedene Keys insgesamt?

Statistische Übersicht: welcher Key kommt wie häufig vor? Hierfür wird ein Dictionary benötigt, das jedem Key den Zähler zuweist.

In [ ]:
keyDict = {}

for elem in pubList:
    for key in elem.keys():
        if key not in keyDict: # wenn noch nicht gesehen...
            keyDict[key] = 1 # ... Initialwert 1
        else:
            keyDict[key] += 1 # sonst den bisher zugewiesenen Wert um 1 erhöhen
keyDict

### Alternative XML-Erzeugung: ElementTree

Das Python-Modul [ElementTree](https://docs.python.org/3.8/library/xml.etree.elementtree.html#) eignet sich zum Erzeugen wie auch zum Lesen von XML. Insbesondere zum Lesen ist es quasi unabdinglich, ein Modul wie dieses zu benutzen, da es aufgrund der hierarchischen Struktur des Formates manuell ungleich schwieriger ist.

In [ ]:
import xml.etree.ElementTree as ET

Wir verwenden `as ET` beim Import, um für zukünftige Aufrufe nur dieses Kürzel verwenden zu müssen, statt des ausführlichen Namens `ElementTree`.

Die wesentlichen Methoden des Moduls zum Erzeugen einer neuen Struktur sind `Element` und `Subelement`. Der wesentliche Parameter für `Element` ist der Tag-Bezeichner. `SubElement` bekommt als zusätzliches Argument das Eltern-Element.

Hier ein Beispiel, um eine Teilstruktur zu erzeugen:

In [ ]:
publications = ET.Element('publications') # einfaches Element
publication = ET.SubElement(publications, 'publication') # Kindelement des root-Elements
title = ET.SubElement(publication, 'title') # Kindelement des publication-Elements
abstract = ET.SubElement(publication, 'abstract') # weiteres Kindelement des publication-Elements
ET.dump(publications) # Ausgabe der Struktur

Hier fehlt zunächst noch sämtlicher Inhalt. Textuellen Inhalt setzt man mit der Methode `text` auf ein Element.

In [ ]:
publications = ET.Element('publications') # einfaches Element
publication = ET.SubElement(publications, 'publication') # Kindelement des root-Elements
title = ET.SubElement(publication, 'title') # Kindelement des publication-Elements
title.text = "Ein Titel"
abstract = ET.SubElement(publication, 'abstract') # weiteres Kindelement des publication-Elements
abstract.text = "Ein abstract"
ET.dump(publications)

Um die Struktur in eine Datei zu schreiben, gibt es auch eine einfache Methode: `write`. Hier gibt es viele Parameter zum Steuern der Ausgabe, nachzuschlagen in der [Dokumentation](https://docs.python.org/3.8/library/xml.etree.elementtree.html#xml.etree.ElementTree.ElementTree.write). Am wichtigsten sind natürlich der Dateiname, und ggf. das Encoding (Standard ist hier ASCII). Der dritte Parameter ist sinnvoll, um die XML-Deklaration hinzuzufügen, um komplett valides XML zu erhalten.

In [ ]:
xml = ET.ElementTree(publications)
xml.write("output/example.xml","utf-8", True)

Mit diesen Informationen ließe sich nun eine Lösung für die Aufgabe mit ElementTree gestalten, die uns das manuelle Schreiben der Start- und Endtags ebenso wie das Escapen von Spezialzeichen erspart:

In [ ]:
publications = ET.Element("publications")
for index, data in enumerate(pubList):
    publication = ET.Element("publication")
    id = ET.SubElement(publication, "id")
    id.text = str(index+1) # index+1, weil der Index sonst bei 0 startet
    
    if "label" in data:
        title = ET.SubElement(publication,"title")
        title.text = data.get('label')
    if "authors" in data:
        authors = ET.SubElement(publication,"authors")
        for i in range(len(data.get("authors"))):
            author = ET.SubElement(authors,"author")
            firstName = ET.SubElement(author,"firstName")
            firstName.text = data.get("authors")[i].get('first') 
            lastName = ET.SubElement(author,"lastName")
            lastName.text = data.get("authors")[i].get('last')
    if "abstract" in data:
        abstract = ET.SubElement(publication,"abstract")
        abstract.text = data.get('abstract')
    if "publisher" in data:
        publisher = ET.SubElement(publication,"publisher")
        publisher.text = data.get('publisher')
    if "year" in data:
        year = ET.SubElement(publication,"year")
        year.text = data.get('year')
    if "pub-type" in data:
        pubtype = ET.SubElement(publication,"type")
        pubtype.text = data.get('pub-type')   

    publications.append(publication)

xml = ET.ElementTree(publications)
xml.write("output/bibsonomy.xml","utf-8", True)

---

## Generieren des Solr-Exportformats

Solr akzeptiert bereits JSON als Importformat, daher könnten wir auch beinahe den Datensatz wie er jetzt ist übergeben. Selbst eine ID ist darin bereits vorhanden (es handelt sich um die eindeutige bibsonomy URL zum Datensatz).

Wir wollen allerdings die Daten vorher noch etwas bereinigen, insbesondere unnötige Felder weglassen und Felder mit schlechten Bezeichnungen umbenennen ('label' -> 'title').

In [ ]:
for item in pubList:
    item['title'] = item.pop('label') # Key umbenennen
    item.pop('intraHash', None) # alternativ: löschen mit del
    #del item['intraHash']
    item.pop('interHash', None)
    item.pop('user', None)
    item.pop('dnbtitleid', None)
    item.pop('bibtexKey', None)    

In [ ]:
#Ausgabe des Ergebnisses zum Prüfen
print(json.dumps(pubList, indent=4))

In [ ]:
with open('output/bibsonomy_to_solr.json', 'w', encoding="utf-8") as outfile:
    json.dump(pubList, outfile, indent=4)

---

Zur Frage: wie könnte ich viele Keys auf einmal löschen, mithilfe der erstellten Statistik über die Vorkommen der Keys?

Das ließe sich über Iteration sowohl über die Publikationsliste als auch die Keys lösen. Wir können hierbei nicht direkt aus dem jeweiligen Item löschen, da man während der Iteration die Daten nicht verändern darf.

In [ ]:
cleanedList = [] # neue Liste für die bereinigten Items

for item in pubList:
    newItem = {} # neues Item
    for key in item.keys():
        if keyDict[key] > 100:
            newItem[key] = item[key] # der Inhalt wird nur ins neue Item aufgenommen, wenn der Key häufig genug vorkommt
    cleanedList.append(newItem) # neues Item der Liste hinzufügen
    
cleanedList